# Google Colab initialization

In [1]:
GOOGLE_COLAB = True
# GOOGLE_COLAB = False

In [2]:
# if GOOGLE_COLAB == True:
#     !pip install pyyaml==5.1

#     import torch
#     TORCH_VERSION = ".".join(torch.__version__.split(".")[:2])
#     CUDA_VERSION = torch.__version__.split("+")[-1]
#     print("torch: ", TORCH_VERSION, "; cuda: ", CUDA_VERSION)
#     # Install detectron2 that matches the above pytorch version
#     # See https://detectron2.readthedocs.io/tutorials/install.html for instructions
#     !pip install detectron2 -f https://dl.fbaipublicfiles.com/detectron2/wheels/$CUDA_VERSION/torch$TORCH_VERSION/index.html > /dev/null 
#     # If there is not yet a detectron2 release that matches the given torch + CUDA version, you need to install a different pytorch.

#     exit(0)  # After installation, you may need to "restart runtime" in Colab. This line can also restart runtime

In [3]:
if GOOGLE_COLAB == True:
    from google.colab import drive
    drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [4]:
# if GOOGLE_COLAB == True:
#     !ls '/content/drive/MyDrive/18744/data/Cityscapes/'

#     !mkdir '/content/data/'
#     !mkdir '/content/data/Cityscapes/'

#     !unzip -n '/content/drive/MyDrive/18744/data/Cityscapes/gtFine_trainvaltest.zip' -d '/content/data/Cityscapes/' > /dev/null
#     !unzip -n '/content/drive/MyDrive/18744/data/Cityscapes/leftImg8bit_trainvaltest.zip' -d '/content/data/Cityscapes/' > /dev/null
#     !tar -xf '/content/drive/MyDrive/18744/data/Cityscapes/mapped_labels.tar' --skip-old-files --directory '/content/data/Cityscapes/'

#     !ls '/content/drive/MyDrive/18744/data/Carla/packaging/'

#     !mkdir '/content/data/'
#     !mkdir '/content/data/Carla/'
#     !mkdir '/content/data/Carla/packaging/'

#     !unzip -n '/content/drive/MyDrive/18744/data/Carla/packaging/package2.zip' -d '/content/data/Carla/packaging/' > /dev/null
#     !unzip -n '/content/drive/MyDrive/18744/data/Carla/packaging/package3.zip' -d '/content/data/Carla/packaging/' > /dev/null
#     !unzip -n '/content/drive/MyDrive/18744/data/Carla/packaging/package4.zip' -d '/content/data/Carla/packaging/' > /dev/null
#     !unzip -n '/content/drive/MyDrive/18744/data/Carla/packaging/package5.zip' -d '/content/data/Carla/packaging/' > /dev/null
#     !unzip -n '/content/drive/MyDrive/18744/data/Carla/packaging/package6.zip' -d '/content/data/Carla/packaging/' > /dev/null
#     !unzip -n '/content/drive/MyDrive/18744/data/Carla/packaging/package7.zip' -d '/content/data/Carla/packaging/' > /dev/null
#     !unzip -n '/content/drive/MyDrive/18744/data/Carla/packaging/package8.zip' -d '/content/data/Carla/packaging/' > /dev/null
#     !unzip -n '/content/drive/MyDrive/18744/data/Carla/packaging/package9.zip' -d '/content/data/Carla/packaging/' > /dev/null
#     !tar -xf '/content/drive/MyDrive/18744/data/Carla/packaging/semantic_train.tar' --skip-old-files --directory '/content/data/Carla/packaging/'

#     !ls '/content/drive/MyDrive/18744/data/RainAddition/train/'

#     !mkdir '/content/data/'
#     !mkdir '/content/data/RainAddition/'
#     !mkdir '/content/data/RainAddition/train/'

#     !tar -xf '/content/drive/MyDrive/18744/data/RainAddition/train/1.tar' --skip-old-files --directory '/content/data/RainAddition/train/'
#     !tar -xf '/content/drive/MyDrive/18744/data/RainAddition/train/2.tar' --skip-old-files --directory '/content/data/RainAddition/train/'
#     !tar -xf '/content/drive/MyDrive/18744/data/RainAddition/train/3.tar' --skip-old-files --directory '/content/data/RainAddition/train/'

In [5]:
if GOOGLE_COLAB == True:
    !ls '/content/drive/MyDrive/18744/data/'
    %cd '/content/drive/MyDrive/18744/Robust-Vision-in-Rain/Detectron2Predictor/'
    !ls

Carla  Cityscapes  RainAddition
/content/drive/MyDrive/18744/Robust-Vision-in-Rain/Detectron2Predictor
classes.txt			    detectron2_sem_seg_trainer.py
classes.xlsx			    evaluation.xlsx
configs				    __init__.py
data_time_gdrive_vs_local.txt	    output
detectron2_predictor.ipynb	    output_all
detectron2_predictor.py		    __pycache__
detectron2_real_vs_simulated.ipynb  tensorboard.ipynb
detectron2_sem_seg_trainer.ipynb    utilities.py


# Libraries

In [6]:
main_dir = './'
data_dir = '/home/tunx404/Miscellaneous/data/' # Local Jupyter

if GOOGLE_COLAB == True:
    # data_dir = '/content/drive/MyDrive/18744/data/'
    data_dir = '/content/data/'

"""Detectron2 heads

ObjectDetection
SemanticSegmentation
InstanceSegmentation
PanopticSegmentation
"""

from detectron2_predictor import Detectron2Predictor

from utilities import create_file_list, imshow_jupyter
# import carla_converter

import os

import cv2
import matplotlib.pyplot as plt
from PIL import Image

import torch
import numpy as np

# Some basic setup:
# Setup detectron2 logger
import detectron2
from detectron2.utils.logger import setup_logger
setup_logger()

# import some common libraries
import numpy as np
import os, json, cv2, random
if GOOGLE_COLAB:
    from google.colab.patches import cv2_imshow

# import some common detectron2 utilities
from detectron2 import model_zoo
from detectron2.engine import DefaultPredictor
from detectron2.config import get_cfg
from detectron2.utils.visualizer import Visualizer
from detectron2.data import MetadataCatalog, DatasetCatalog

from detectron2.engine import DefaultTrainer, hooks
from detectron2.evaluation import SemSegEvaluator, inference_on_dataset
from detectron2.data import build_detection_test_loader

from tqdm import tqdm

# List images & get annotations

## Carla

In [7]:
data_carla_dir = data_dir + 'Carla/packaging/'

# Dir structure:
# <data_carla_dir>
#     packages2
#     packages3
#     ...

def get_carla_file_list(data_dir, packages=[], levels=[]):
    file_list = []
    
    for package in packages:
        temp_file_name_list, temp_file_path_list = create_file_list(data_dir, package)
        
        for i in range(len(temp_file_name_list)):
            file_name_split = temp_file_name_list[i].split('_')
            if len(file_name_split) == 3: # [id, type, level.png]
                file_id = file_name_split[0]
                level = file_name_split[2].replace('.png', '')
                if level in levels:
                    file_list.append((file_id, level, package))
                    
    print(f'Number of images: {len(file_list)}')
    
    return file_list

data_carla_train_file_list = get_carla_file_list(data_carla_dir, packages=['package2', 'package3', 'package4', 'package5', 'package6', 'package7', 'package9'], levels=['H', 'M', 'S'])
data_carla_val_file_list = get_carla_file_list(data_carla_dir, packages=['package8'], levels=['H', 'M', 'S'])

# # Toy example
# data_carla_train_file_list = data_carla_train_file_list[:10]
# data_carla_val_file_list = data_carla_val_file_list[:10]

def get_carla_dicts(file_list, data_dir, clear=True, rain=True):
    dicts = []
    
    for file in file_list:
        file_id, level, package = file

        image_clear_path = os.path.join(data_dir, package, file_id + '_clear.png')
        image_rain_path = os.path.join(data_dir, package, file_id + '_rain_' + level + '.png')
        # image_semantic_path = os.path.join(data_dir, package, file_id + '_semantic_single.png') # Carla ID
        image_semantic_path = os.path.join(data_dir, package, file_id + '_semantic_train.png') # Train ID

        if clear == True:
            record = {}
            record['file_name'] = image_clear_path
            record['height'] = 720 # shape[0]
            record['width'] = 1280 # shape[1]
            record['image_id'] = file_id + '_clear'
            record['sem_seg_file_name'] = image_semantic_path
            dicts.append(record)
        
        if rain == True:
            record = {}
            record['file_name'] = image_rain_path
            record['height'] = 720 # shape[0]
            record['width'] = 1280 # shape[1]
            record['image_id'] = file_id + '_rain'
            record['sem_seg_file_name'] = image_semantic_path
            dicts.append(record)

    return dicts

print('**************************************************')
print('Carla clear')
def get_carla_clear_train_dicts():
    return get_carla_dicts(data_carla_train_file_list, data_carla_dir, clear=True, rain=False)

def get_carla_clear_val_dicts():
    return get_carla_dicts(data_carla_val_file_list, data_carla_dir, clear=True, rain=False)

for dict in get_carla_clear_train_dicts()[:4]:
    print(dict)
    
print(len(get_carla_clear_train_dicts())) # 2990
print(len(get_carla_clear_val_dicts()))   # 1173

print('\n**************************************************')
print('Carla rain')
def get_carla_rain_train_dicts():
    return get_carla_dicts(data_carla_train_file_list, data_carla_dir, clear=False, rain=True)

def get_carla_rain_val_dicts():
    return get_carla_dicts(data_carla_val_file_list, data_carla_dir, clear=False, rain=True)

for dict in get_carla_rain_train_dicts()[:4]:
    print(dict)
    
print(len(get_carla_rain_train_dicts())) # 2990
print(len(get_carla_rain_val_dicts()))   # 1173

print('\n**************************************************')
print('Carla clear + rain')
def get_carla_all_train_dicts():
    return get_carla_dicts(data_carla_train_file_list, data_carla_dir, clear=True, rain=True)

def get_carla_all_val_dicts():
    return get_carla_dicts(data_carla_val_file_list, data_carla_dir, clear=True, rain=True)

for dict in get_carla_all_train_dicts()[:4]:
    print(dict)
    
print(len(get_carla_all_train_dicts())) # 2990*2 = 5980
print(len(get_carla_all_val_dicts()))   # 1173*2 = 2346

Number of images: 2990
Number of images: 1173
**************************************************
Carla clear
{'file_name': '/content/data/Carla/packaging/package2/1647144040.0070002_clear.png', 'height': 720, 'width': 1280, 'image_id': '1647144040.0070002_clear', 'sem_seg_file_name': '/content/data/Carla/packaging/package2/1647144040.0070002_semantic_train.png'}
{'file_name': '/content/data/Carla/packaging/package2/1647144051.8223236_clear.png', 'height': 720, 'width': 1280, 'image_id': '1647144051.8223236_clear', 'sem_seg_file_name': '/content/data/Carla/packaging/package2/1647144051.8223236_semantic_train.png'}
{'file_name': '/content/data/Carla/packaging/package2/1647144063.4320533_clear.png', 'height': 720, 'width': 1280, 'image_id': '1647144063.4320533_clear', 'sem_seg_file_name': '/content/data/Carla/packaging/package2/1647144063.4320533_semantic_train.png'}
{'file_name': '/content/data/Carla/packaging/package2/1647144076.9520714_clear.png', 'height': 720, 'width': 1280, 'image_i

## Cityscapes

In [8]:
data_dir_cityscapes = data_dir + 'Cityscapes/leftImg8bit/train/'
data_dir_rain_addition = data_dir + 'RainAddition/train/'
# anno_dir_cityscapes = data_dir + 'Cityscapes/gtFine/train/'
anno_dir_cityscapes = data_dir + 'Cityscapes/mapped_labels/train/'

city_name_list, _ = create_file_list(data_dir_cityscapes)
print(f'Number of cities: {len(city_name_list)}')

def get_cityscapes_file_list(data_dir, cities=[], levels=[]):
    file_list = []
    
    for city in cities:
        temp_file_name_list, _ = create_file_list(data_dir, city)
        
        for file_name in temp_file_name_list:
            file_list.append((file_name, city))
                    
    print(f'Number of images: {len(file_list)}')
    
    return file_list

data_cityscapes_train_file_list = get_cityscapes_file_list(data_dir_cityscapes, cities=city_name_list[:13], levels=['H', 'M', 'S'])
data_cityscapes_val_file_list   = get_cityscapes_file_list(data_dir_cityscapes, cities=city_name_list[13:], levels=['H', 'M', 'S'])

# # Toy example
# data_cityscapes_train_file_list = data_cityscapes_train_file_list[:10]
# data_cityscapes_val_file_list = data_cityscapes_val_file_list[:10]

def get_cityscapes_dicts(file_list, data_dir_main, data_dir_rain, anno_dir, clear=True, rain=True, levels=[]):
    dicts = []
    
    # for file in file_list:
    for index, file in enumerate(file_list):
        file_name, city = file

        image_clear_path = os.path.join(data_dir_main, city, file_name)
        
        file_name_split = file_name.split('_')
        # anno_name = file_name_split[0] + '_' + file_name_split[1] + '_' + file_name_split[2] + '_gtFine_labelIds.png' # Default
        image_id = file_name_split[0] + '_' + file_name_split[1] + '_' + file_name_split[2]
        anno_name = image_id + '_train.png' # Mapped
        image_semantic_path = os.path.join(anno_dir, city, anno_name)

        if clear == True:
            record = {}
            record['file_name'] = image_clear_path
            record['height'] = 1024 # shape[0]
            record['width'] = 2048 # shape[1]
            record['image_id'] = image_id + '_clear'
            record['sem_seg_file_name'] = image_semantic_path
            dicts.append(record)

        if rain == True:
            num_levels = len(levels)
            # for level in levels: # All 3 levels
            for level in [levels[index%num_levels]]: # 1 level
                image_rain_name = image_id + '_' + level + '.png'
                image_rain_path = os.path.join(data_dir_rain, city, image_rain_name)

                record = {}
                record['file_name'] = image_rain_path
                record['height'] = 1024
                record['width'] = 2048
                record['image_id'] = image_id + '_rain'
                record['sem_seg_file_name'] = image_semantic_path
                dicts.append(record)

    return dicts

print('**************************************************')
print('Cityscapes clear')
def get_cityscapes_clear_train_dicts():
    return get_cityscapes_dicts(data_cityscapes_train_file_list, data_dir_cityscapes, data_dir_rain_addition, anno_dir_cityscapes, clear=True, rain=False, levels=['H', 'M', 'S'])

def get_cityscapes_clear_val_dicts():
    return get_cityscapes_dicts(data_cityscapes_val_file_list, data_dir_cityscapes, data_dir_rain_addition, anno_dir_cityscapes, clear=True, rain=False, levels=['H', 'M', 'S'])

for dict in get_cityscapes_clear_train_dicts()[:4]:
    print(dict)
    
print(len(get_cityscapes_clear_train_dicts())) # 2276
print(len(get_cityscapes_clear_val_dicts()))   # 699

print('\n**************************************************')
print('Cityscapes rain')
def get_cityscapes_rain_train_dicts():
    return get_cityscapes_dicts(data_cityscapes_train_file_list, data_dir_cityscapes, data_dir_rain_addition, anno_dir_cityscapes, clear=False, rain=True, levels=['H', 'M', 'S'])

def get_cityscapes_rain_val_dicts():
    return get_cityscapes_dicts(data_cityscapes_val_file_list, data_dir_cityscapes, data_dir_rain_addition, anno_dir_cityscapes, clear=False, rain=True, levels=['H', 'M', 'S'])

for dict in get_cityscapes_rain_train_dicts()[:4]:
    print(dict)
    
print(len(get_cityscapes_rain_train_dicts())) # 2276
print(len(get_cityscapes_rain_val_dicts()))   # 699

print('\n**************************************************')
print('Cityscapes clear + rain')
def get_cityscapes_all_train_dicts():
    return get_cityscapes_dicts(data_cityscapes_train_file_list, data_dir_cityscapes, data_dir_rain_addition, anno_dir_cityscapes, clear=True, rain=True, levels=['H', 'M', 'S'])

def get_cityscapes_all_val_dicts():
    return get_cityscapes_dicts(data_cityscapes_val_file_list, data_dir_cityscapes, data_dir_rain_addition, anno_dir_cityscapes, clear=True, rain=True, levels=['H', 'M', 'S'])

for dict in get_cityscapes_all_train_dicts()[:4]:
    print(dict)
    
print(len(get_cityscapes_all_train_dicts())) # 2276*2 = 4552
print(len(get_cityscapes_all_val_dicts()))   # 699*2  = 1398

Number of cities: 18
Number of images: 2276
Number of images: 699
**************************************************
Cityscapes clear
{'file_name': '/content/data/Cityscapes/leftImg8bit/train/aachen/aachen_000000_000019_leftImg8bit.png', 'height': 1024, 'width': 2048, 'image_id': 'aachen_000000_000019_clear', 'sem_seg_file_name': '/content/data/Cityscapes/mapped_labels/train/aachen/aachen_000000_000019_train.png'}
{'file_name': '/content/data/Cityscapes/leftImg8bit/train/aachen/aachen_000001_000019_leftImg8bit.png', 'height': 1024, 'width': 2048, 'image_id': 'aachen_000001_000019_clear', 'sem_seg_file_name': '/content/data/Cityscapes/mapped_labels/train/aachen/aachen_000001_000019_train.png'}
{'file_name': '/content/data/Cityscapes/leftImg8bit/train/aachen/aachen_000002_000019_leftImg8bit.png', 'height': 1024, 'width': 2048, 'image_id': 'aachen_000002_000019_clear', 'sem_seg_file_name': '/content/data/Cityscapes/mapped_labels/train/aachen/aachen_000002_000019_train.png'}
{'file_name': 

## Combined

In [9]:
print('**************************************************')
print('Combined clear')
def get_combined_clear_train_dicts():
    return get_carla_clear_train_dicts() + get_cityscapes_clear_train_dicts()

def get_combined_clear_val_dicts():
    return get_carla_clear_val_dicts() + get_cityscapes_clear_val_dicts()

for dict in get_combined_clear_train_dicts()[:4]:
    print(dict)
for dict in get_combined_clear_train_dicts()[-4:]:
    print(dict)
    
print(len(get_combined_clear_train_dicts())) # 2990 + 2276 = 5266
print(len(get_combined_clear_val_dicts()))   # 1173 + 699  = 1872

print('\n**************************************************')
print('Combined rain')
def get_combined_rain_train_dicts():
    return get_carla_rain_train_dicts() + get_cityscapes_rain_train_dicts()

def get_combined_rain_val_dicts():
    return get_carla_rain_val_dicts() + get_cityscapes_rain_val_dicts()

for dict in get_combined_rain_train_dicts()[:4]:
    print(dict)
for dict in get_combined_rain_train_dicts()[-4:]:
    print(dict)
    
print(len(get_combined_rain_train_dicts())) # 2990 + 2276 = 5266
print(len(get_combined_rain_val_dicts()))   # 1173 + 699  = 1872

print('\n**************************************************')
print('Combined clear + rain')
def get_combined_all_train_dicts():
    return get_carla_all_train_dicts() + get_cityscapes_all_train_dicts()

def get_combined_all_val_dicts():
    return get_carla_all_val_dicts() + get_cityscapes_all_val_dicts()

for dict in get_combined_all_train_dicts()[:4]:
    print(dict)
for dict in get_combined_all_train_dicts()[-4:]:
    print(dict)

print(len(get_combined_all_train_dicts())) # 5266*2 = 10532
print(len(get_combined_all_val_dicts()))   # 1872*2 = 3744

**************************************************
Combined clear
{'file_name': '/content/data/Carla/packaging/package2/1647144040.0070002_clear.png', 'height': 720, 'width': 1280, 'image_id': '1647144040.0070002_clear', 'sem_seg_file_name': '/content/data/Carla/packaging/package2/1647144040.0070002_semantic_train.png'}
{'file_name': '/content/data/Carla/packaging/package2/1647144051.8223236_clear.png', 'height': 720, 'width': 1280, 'image_id': '1647144051.8223236_clear', 'sem_seg_file_name': '/content/data/Carla/packaging/package2/1647144051.8223236_semantic_train.png'}
{'file_name': '/content/data/Carla/packaging/package2/1647144063.4320533_clear.png', 'height': 720, 'width': 1280, 'image_id': '1647144063.4320533_clear', 'sem_seg_file_name': '/content/data/Carla/packaging/package2/1647144063.4320533_semantic_train.png'}
{'file_name': '/content/data/Carla/packaging/package2/1647144076.9520714_clear.png', 'height': 720, 'width': 1280, 'image_id': '1647144076.9520714_clear', 'sem_seg_fi

# Convert labels to train IDs
Execute only once

In [10]:
# %cd '/home/tunx404/Miscellaneous/data/Cityscapes/mapped_labels/train/'
# for city_name in city_name_list:
#     !mkdir $city_name

In [11]:
map_cityscapes_id_to_carla_id = { 
     0:  0, # unlabeled           
     1:  0, # ego vehicle         
     2:  0, # rectification border
     3:  0, # out of roi          
     4: 19, # static              
     5: 20, # dynamic             
     6: 14, # ground              
     7:  7, # road                
     8:  8, # sidewalk            
     9:  0, # parking             
    10: 16, # rail track          
    11:  1, # building            
    12: 11, # wall                
    13:  2, # fence               
    14: 17, # guard rail          
    15: 15, # bridge              
    16:  0, # tunnel              
    17:  5, # pole                
    18:  0, # polegroup           
    19: 18, # traffic light       
    20: 12, # traffic sign        
    21:  9, # vegetation          
    22: 22, # terrain             
    23: 13, # sky                 
    24:  4, # person              
    25:  4, # rider               
    26: 10, # car                 
    27: 10, # truck               
    28: 10, # bus                 
    29:  0, # caravan             
    30:  0, # trailer             
    31: 10, # train               
    32: 10, # motorcycle          
    33: 10, # bicycle             
    -1:  0  # license plate       
}

map_carla_id_to_train_id = {
    0 :  0, # Unlabeled    # Unlabeled    # (0, 0, 0)
    1 :  1, # Building     # Building     # (70, 70, 70)
    2 :  2, # Fence        # Fence        # (100, 40, 40)
    3 :  0, # Other        #
    4 :  3, # Pedestrian   # Pedestrian   # (220, 20, 60)
    5 :  4, # Pole         # Pole         # (153, 153, 153)
    6 :  5, # RoadLine     #
    7 :  5, # Road         # Road         # (128, 64, 128)
    8 :  6, # SideWalk     # SideWalk     # (244, 35, 232)
    9 :  7, # Vegetation   # Vegetation   # (107, 142, 35)
    10:  8, # Vehicles     # Vehicles     # (0, 0, 142)
    11:  9, # Wall         # Wall         # (102, 102, 156)
    12: 10, # TrafficSign  # TrafficSign  # (220, 220, 0)
    13: 11, # Sky          # Sky          # (70, 130, 180)
    14:  0, # Ground       #
    15:  0, # Bridge       #
    16:  0, # RailTrack    #
    17:  0, # GuardRail    #
    18: 12, # TrafficLight # TrafficLight # (250, 170, 30)
    19:  0, # Static       #
    20:  0, # Dynamic      #
    21:  0, # Water        #
    22: 13  # Terrain      # Terrain      # (145, 170, 100)
}

def encode_labels(mask, map):
    label_mask = np.zeros_like(mask)
    for k in map:
        label_mask[mask == k] = map[k]
    return label_mask

def convert_carla(file_list, data_dir):
    for i in tqdm(range(len(file_list))):
        file_id, level, package = file_list[i]
        image_semantic_path = os.path.join(data_dir, package, file_id + '_semantic.png')
        image_semantic = cv2.imread(image_semantic_path)[:, :, 2] # HxWxC, BGR
        image_semantic = encode_labels(image_semantic, map=map_carla_id_to_train_id)
        
        # print(image_semantic_path)
        # imshow_jupyter(image_semantic)
        
        output_file_name = os.path.join(data_dir, package, file_id + '_semantic_train.png')
        if not os.path.exists(output_file_name):
            # print(output_file_name)
            cv2.imwrite(output_file_name, image_semantic)
            
def convert_cityscapes(file_list, anno_dir):
    for i in tqdm(range(len(file_list))):
        file_name, city = file_list[i]
        file_name_split = file_name.split('_')
        
        output_dir = '/home/tunx404/Miscellaneous/data/Cityscapes/mapped_labels/train/'
        output_file_name = file_name_split[0] + '_' + file_name_split[1] + '_' + file_name_split[2] + '_train.png'
        output_file_name = os.path.join(output_dir, city, output_file_name)
        
        if not os.path.exists(output_file_name):
            anno_name = file_name_split[0] + '_' + file_name_split[1] + '_' + file_name_split[2] + '_gtFine_labelIds.png'
            image_semantic_path = os.path.join(anno_dir, city, anno_name)
            image_semantic = cv2.imread(image_semantic_path)[:, :, 0] # HxWxC, BGR
            image_semantic = encode_labels(image_semantic, map=map_cityscapes_id_to_carla_id)
            image_semantic = encode_labels(image_semantic, map=map_carla_id_to_train_id)
        
            # print(image_semantic)
            # imshow_jupyter(image_semantic)
            
            # print(output_file_name)
            cv2.imwrite(output_file_name, image_semantic)

# convert_carla(data_carla_train_file_list, data_carla_dir)
# convert_carla(data_carla_val_file_list, data_carla_dir)

# convert_cityscapes(data_cityscapes_train_file_list, anno_dir_cityscapes)
# convert_cityscapes(data_cityscapes_val_file_list, anno_dir_cityscapes)

# Datasets

In [12]:
class Detectron2CustomDataset:
    train_classes = ['Unlabeled', 'Building', 'Fence', 'Pedestrian', 'Pole', 'Road', 'SideWalk', 'Vegetation', 'Vehicles', 'Wall', 'TrafficSign', 'Sky', 'TrafficLight', 'Terrain']
    train_colors = [(0, 0, 0), (70, 70, 70), (100, 40, 40), (220, 20, 60), (153, 153, 153), (128, 64, 128), (244, 35, 232), (107, 142, 35), (0, 0, 142), (102, 102, 156), (220, 220, 0), (70, 130, 180), (250, 170, 30), (145, 170, 100)]
    
    def __init__(self, train_dataset_name, val_dataset_name, get_train_dicts_fn, get_val_dicts_fn, classes=None, colors=None, ignore_label=0):
        
        if classes is None:
            self.classes = self.train_classes
        if colors is None:
            self.colors = self.train_colors
        print(f'Number of classes: {len(self.classes)}')
        
        self.train_dataset_name = train_dataset_name
        self.val_dataset_name = val_dataset_name
        
        self.get_train_dicts_fn = get_train_dicts_fn
        self.get_val_dicts_fn = get_val_dicts_fn

        print(f'Number of train images: {len(self.get_train_dicts_fn())}')
        print(f'Number of val images:   {len(self.get_val_dicts_fn())}')
        
        DatasetCatalog.register(self.train_dataset_name, self.get_train_dicts_fn)
        DatasetCatalog.register(self.val_dataset_name, self.get_val_dicts_fn)

        MetadataCatalog.get(self.train_dataset_name).stuff_classes = self.classes
        MetadataCatalog.get(self.train_dataset_name).stuff_colors = self.colors
        MetadataCatalog.get(self.train_dataset_name).ignore_label = 0

        MetadataCatalog.get(self.val_dataset_name).stuff_classes = self.classes
        MetadataCatalog.get(self.val_dataset_name).stuff_colors = self.colors
        MetadataCatalog.get(self.val_dataset_name).ignore_label = ignore_label
        
    def visualize_train_dataset(self, num_samples=1, size=(12, 6)):
        train_metadata = MetadataCatalog.get(self.train_dataset_name)
        data_train_dicts = self.get_train_dicts_fn()

        for file_dict in random.sample(data_train_dicts, num_samples):
            image = cv2.imread(file_dict['file_name'])
            visualizer = Visualizer(image[:, :, ::-1], metadata=train_metadata, scale=0.5)
            output = visualizer.draw_dataset_dict(file_dict)
            image = cv2.cvtColor(output.get_image()[:, :, ::-1], cv2.COLOR_BGR2RGB)
            imshow_jupyter(image, size=size)

    def visualize_val_dataset(self, predictor, num_samples=1, size=(12, 6)):
        val_metadata = MetadataCatalog.get(self.val_dataset_name)
        data_val_dicts = self.get_val_dicts_fn()

        for file_dict in random.sample(data_val_dicts, num_samples):
            image = cv2.imread(file_dict['file_name'])
            visualizer = Visualizer(image[:, :, ::-1], metadata=val_metadata, scale=0.5)
            
            print('Ground truth')
            output = visualizer.draw_dataset_dict(file_dict)
            target_image = cv2.cvtColor(output.get_image()[:, :, ::-1], cv2.COLOR_BGR2RGB)
            imshow_jupyter(target_image)
            
            print('Predicted')
            outputs = predictor(image)
            sem_seg = torch.argmax(outputs['sem_seg'], dim=0)
            output = visualizer.draw_sem_seg(sem_seg.to('cpu'))
            predicted_image = cv2.cvtColor(output.get_image()[:, :, ::-1], cv2.COLOR_BGR2RGB)
            imshow_jupyter(predicted_image)
            
            print()

DatasetCatalog.clear()

# print('Carla dataset')
# carla_all_dataset = Detectron2CustomDataset('carla_train', 'carla_val', get_carla_train_dicts, get_carla_val_dicts)
# # carla_all_dataset.visualize_train_dataset(num_samples=1, size=(20, 10))
            
# print('\nCityscapes dataset')
# cityscapes_all_dataset = Detectron2CustomDataset('cityscapes_train', 'cityscapes_val', get_cityscapes_train_dicts, get_cityscapes_val_dicts)
# # cityscapes_dataset.visualize_train_dataset(num_samples=1, size=(20, 10))
            
print('\nCombined clear dataset')
combined_clear_dataset = Detectron2CustomDataset('combined_clear_train', 'combined_clear_val', get_combined_clear_train_dicts, get_combined_clear_val_dicts)
# combined_clear_dataset.visualize_train_dataset(num_samples=4, size=(20, 10))
            
print('\nCombined rain dataset')
combined_rain_dataset = Detectron2CustomDataset('combined_rain_train', 'combined_rain_val', get_combined_rain_train_dicts, get_combined_rain_val_dicts)
# combined_rain_dataset.visualize_train_dataset(num_samples=4, size=(20, 10))
            
print('\nCombined clear + rain dataset')
combined_all_dataset = Detectron2CustomDataset('combined_all_train', 'combined_all_val', get_combined_all_train_dicts, get_combined_all_val_dicts)
# combined_all_dataset.visualize_train_dataset(num_samples=4, size=(20, 10))


Combined clear dataset
Number of classes: 14
Number of train images: 5266
Number of val images:   1872

Combined rain dataset
Number of classes: 14
Number of train images: 5266
Number of val images:   1872

Combined clear + rain dataset
Number of classes: 14
Number of train images: 10532
Number of val images:   3744


# Trainer class

In [13]:
class MyTrainer(DefaultTrainer):
    @classmethod
    def build_evaluator(cls, cfg, dataset_name, output_folder=None):
        if output_folder is None:
            output_folder = os.path.join(cfg.OUTPUT_DIR, 'inference')
        return SemSegEvaluator(dataset_name, output_folder)

class Detectron2Trainer:
    def __init__(self, train_dataset_name, val_dataset_name, output_folder):
        
        config_path = 'configs/Misc/semantic_R_50_FPN_1x.yaml'
        # model_path = 'https://dl.fbaipublicfiles.com/detectron2/ImageNetPretrained/MSRA/R-50.pkl'

        # All configs: https://detectron2.readthedocs.io/en/latest/modules/config.html
        
        self.train_dataset_name = train_dataset_name
        self.val_dataset_name = val_dataset_name

        self.cfg = get_cfg()
        self.cfg.merge_from_file(config_path)

        # self.cfg.MODEL.WEIGHTS = model_path
        self.cfg.MODEL.DEVICE = 'cuda'
        # self.cfg.MODEL.DEVICE = 'cpu'

        self.cfg.DATASETS.TRAIN = (self.train_dataset_name,)
        # self.cfg.DATASETS.TEST = (self.train_dataset_name,)
        self.cfg.DATASETS.TEST = (self.val_dataset_name,)

        self.cfg.DATALOADER.NUM_WORKERS = 2

        self.cfg.INPUT.MIN_SIZE_TRAIN = 720
        self.cfg.INPUT.MAX_SIZE_TRAIN = 2048
        self.cfg.INPUT.MIN_SIZE_TEST = 720
        self.cfg.INPUT.MAX_SIZE_TEST = 2048

        # Number of images per batch across all machines. This is also the number
        # of training images per step (i.e. per iteration).
        self.cfg.SOLVER.IMS_PER_BATCH = 8
        self.cfg.SOLVER.BASE_LR = 0.01
        self.cfg.SOLVER.MAX_ITER = 20000
        self.cfg.SOLVER.GAMMA = 0.1
        # The iteration number to decrease learning rate by GAMMA.
        self.cfg.SOLVER.STEPS = (10000, 15000, 18000, 19000)
        # Save a checkpoint after every this number of iterations
        self.cfg.SOLVER.CHECKPOINT_PERIOD = 1000

        self.cfg.TEST.EVAL_PERIOD = 1000

        self.cfg.MODEL.SEM_SEG_HEAD.IGNORE_VALUE = 0
        classes = MetadataCatalog.get(self.train_dataset_name).stuff_classes
        self.cfg.MODEL.SEM_SEG_HEAD.NUM_CLASSES = len(classes)
        
        # Directory where output files are written
        self.cfg.OUTPUT_DIR = output_folder
        os.makedirs(self.cfg.OUTPUT_DIR, exist_ok=True)

        # self.trainer = DefaultTrainer(self.cfg)
        self.trainer = MyTrainer(self.cfg)
        
        self.evaluator = None
        self.predictor = None
        
    def load(self):
        self.trainer.resume_or_load(resume=True)
        
    def train(self):
        self.trainer.train()
        
    def get_predictor(self, output_folder=None):
        if output_folder is None:
            output_folder = self.cfg.OUTPUT_DIR
        last_checkpoint = 'model_final.pth'
        with open(os.path.join(output_folder, 'last_checkpoint')) as file:
            last_checkpoint = file.read()
        print('Last checkpoint: ' + last_checkpoint)
            
        self.cfg.MODEL.WEIGHTS = os.path.join(self.cfg.OUTPUT_DIR, last_checkpoint)
        self.predictor = DefaultPredictor(self.cfg)
        
    def test(self, output_folder=None):
        if output_folder is None:
            output_folder = self.cfg.OUTPUT_DIR
        self.get_predictor(output_folder)
        self.evaluator = SemSegEvaluator(self.val_dataset_name, output_dir=os.path.join(output_folder, 'inference'))
        self.trainer.test(self.cfg, self.predictor.model, self.evaluator)

# Train

In [14]:
# trainer_all = Detectron2Trainer('combined_all_train', 'combined_all_val', output_folder='./output_all')
# trainer_all.load()
# trainer_all.train()
# trainer_all.test()

In [ ]:
trainer_clear = Detectron2Trainer('combined_clear_train', 'combined_clear_val', output_folder='./output_clear')
trainer_clear.load()
trainer_clear.train()
trainer_clear.test()

[04/15 17:07:03 d2.engine.defaults]: Model:
SemanticSegmentor(
  (backbone): FPN(
    (fpn_lateral2): Conv2d(256, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output2): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral3): Conv2d(512, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output3): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral4): Conv2d(1024, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output4): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral5): Conv2d(2048, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output5): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (top_block): LastLevelMaxPool()
    (bottom_up): ResNet(
      (stem): BasicStem(
        (conv1): Conv2d(
          3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False
          (norm): FrozenBatchNorm2d(num_features=64, eps=1e-05)
        )
      )
      (r

Some model parameters or buffers are not found in the checkpoint:
backbone.fpn_lateral2.{bias, weight}
backbone.fpn_lateral3.{bias, weight}
backbone.fpn_lateral4.{bias, weight}
backbone.fpn_lateral5.{bias, weight}
backbone.fpn_output2.{bias, weight}
backbone.fpn_output3.{bias, weight}
backbone.fpn_output4.{bias, weight}
backbone.fpn_output5.{bias, weight}
sem_seg_head.p2.0.norm.{bias, weight}
sem_seg_head.p2.0.weight
sem_seg_head.p3.0.norm.{bias, weight}
sem_seg_head.p3.0.weight
sem_seg_head.p4.0.norm.{bias, weight}
sem_seg_head.p4.0.weight
sem_seg_head.p4.2.norm.{bias, weight}
sem_seg_head.p4.2.weight
sem_seg_head.p5.0.norm.{bias, weight}
sem_seg_head.p5.0.weight
sem_seg_head.p5.2.norm.{bias, weight}
sem_seg_head.p5.2.weight
sem_seg_head.p5.4.norm.{bias, weight}
sem_seg_head.p5.4.weight
sem_seg_head.predictor.{bias, weight}
The checkpoint state_dict contains keys that are not used by the model:
  fc1000.{bias, weight}
  stem.conv1.bias


[04/15 17:07:11 d2.engine.train_loop]: Starting training from iteration 0


/usr/local/lib/python3.7/dist-packages/detectron2/structures/image_list.py:88: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor').
  max_size = (max_size + (stride - 1)) // stride * stride


[04/15 17:07:42 d2.utils.events]:  eta: 8:11:57  iter: 19  total_loss: 5.526  time: 1.4971  data_time: 0.4945  lr: 0.00019981  max_mem: 8441M
[04/15 17:08:12 d2.utils.events]:  eta: 8:16:11  iter: 39  total_loss: 1.128  time: 1.5174  data_time: 0.4929  lr: 0.00039961  max_mem: 8441M


In [ ]:
%load_ext tensorboard
%tensorboard --logdir output_all

In [ ]:
%load_ext tensorboard
%tensorboard --logdir output_clear

# Evaluate

In [ ]:
# trainer_all = Detectron2Trainer('combined_all_train', 'combined_all_val', output_folder='./output_all')
# trainer_all.test()

In [ ]:
# trainer_rain = Detectron2Trainer('combined_rain_train', 'combined_rain_val', output_folder='./output_all')
# trainer_rain.test()

In [ ]:
# trainer_clear = Detectron2Trainer('combined_clear_train', 'combined_clear_val', output_folder='./output_all')
# trainer_clear.test()

In [ ]:
# trainer_rain.get_predictor()
# combined_rain_dataset.visualize_val_dataset(trainer_rain.predictor, num_samples=5)